# Sparkfy Database in Cassandra

A fictitious company called Sparkify wants to analyze their music streaming app data and better undertand who are their users and what they are listening to. In this projet, the following steps will be followed:

- 1 Processing the music, artist and user library containing app data and create a new csv with a sample of the processed data
- 2 Creating the tables optimized to answer the business questions
- 3 Inserting the data in the correspoding table
- 4 Selecting the data in the tables to evaluate the complete process
- 5 Droping the tables and closing connection

## 1 Processing the music, artist and user library containing app data and create a new csv with a sample of the processed data

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# original data total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### The csv sample event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## 2 Creating the tables optimized to answer the business questions

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkfy 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to sparkfy
try:
    session.set_keyspace('sparkfy')
except Exception as e:
    print(e)

### We want to answer the followind business questions:

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
`SELECT artist_name, song_title, song_length FROM sessions WHERE session_id = 338 AND item_in_session = 4`

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
`SELECT artist_name, song_title (sorted by item_in_session), user_first_name, user_last_name FROM user_library WHERE user_id = 10 AND session_id = 182`

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
`SELECT user_first_name, user_last_name FROM music_library WHERE song_title = 'All Hands Against His Own'`


In [8]:
# -------------------------------------------------------------------------------------------------------------- #
# In this scenario, the best approach is to create a table for each query, giving that they are filtering and \
# ordering the data in a different way. Also, in Cassandra it is not a bad strategy to create individual tables \
# for each query.
# -------------------------------------------------------------------------------------------------------------- #

# -------------------------------------------------------------------------------------------------------------- #
# Query 1: "SELECT artist_name, song_title, song_length FROM sessions WHERE session_id=338 AND item_in_session=4"
# PRIMARY KEY: session_id, item_in_session
#
# PARTITIONING KEY: session_id  -> it will always be in the where and hopfully can distribute the data evenly.
#
# CLUSTERING KEY: item_in_session -> it will also be in the where clause and makes the PK unique.
# -------------------------------------------------------------------------------------------------------------- #
query1 = """CREATE TABLE IF NOT EXISTS sessions (
                session_id int, 
                item_in_session int, 
                artist_name text, 
                song_title text,
                song_length float,
                PRIMARY KEY (session_id, item_in_session)
)"""
try:
    session.execute(query1)
except Exception as e:
    print(e)

# -------------------------------------------------------------------------------------------------------------- #
# Query 2: "SELECT artist_name, song_title (sorted by item_in_session), user_first_name, user_last_name FROM \
# user_library WHERE user_id=10 AND session_id=182"
#
# PRIMARY KEY: (user_id, session_id), item_in_session
#
# PARTITIONING KEY: user_id, session_id  -> since the user_id is a 1:N relationship with session_id and both \
# columns will be in the where clause, it is reasonable to use them as a PK, given that the addition of the \
# user_id will not mess with the data distribution through the nodes.
#
# CLUSTERING KEY: item_in_session -> it will sort the songs and make the PK unique
# -------------------------------------------------------------------------------------------------------------- #
query2 = """CREATE TABLE IF NOT EXISTS user_library (
                user_id int,
                session_id int, 
                item_in_session int, 
                artist_name text,
                song_title text,
                user_first_name text,
                user_last_name text,
                PRIMARY KEY ((user_id, session_id), item_in_session)
)"""
try:
    session.execute(query2)
except Exception as e:
    print(e)

# -------------------------------------------------------------------------------------------------------------- #
# Query 3: "SELECT user_first_name, user_last_name FROM music_library WHERE song_title='All Hands Against His Own'"
#
# PRIMARY KEY: song_title, user_id -> the analysis here is just to retrieve unique users for a giving song, \
# the PK needs to be unique only for these two columns
#
# PARTITIONING KEY: song_title  -> since the song_title will always be in the where clause.
#
# CLUSTERING KEY: user_id -> to make users unique for each song
# -------------------------------------------------------------------------------------------------------------- #
query3 = """CREATE TABLE IF NOT EXISTS music_library (
                user_id int, 
                user_first_name text,
                user_last_name text,
                song_title text,
                PRIMARY KEY (song_title, user_id)
)"""
try:
    session.execute(query3)
except Exception as e:
    print(e)

## 3 Inserting the data in the correspoding table

In [9]:
# -------------------------------------------------------------------------------------------------------------- #
# Each row will be read once and inserted in each corresponding table
# -------------------------------------------------------------------------------------------------------------- #

# file with the data to be inserted
file = 'event_datafile_new.csv'

# reading and processing each line of the csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # variables from the csv line
        artist_name = str(line[0])
        user_first_name = str(line[1])
        item_in_session = int(line[3])
        user_last_name = str(line[4])
        song_length = float(line[5])
        session_id = int(line[8])
        song_title = str(line[9])
        user_id = int(line[10])
        
        # QUERY 1 insert statement
        query1 = "INSERT INTO sessions (session_id, item_in_session, artist_name, song_title, song_length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (session_id, item_in_session, artist_name, song_title, song_length))
        
        # QUERY 2 insert statement
        query2 = "INSERT INTO user_library (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name))

        # QUERY 3 insert statement
        query3 = "INSERT INTO music_library (user_id, user_first_name, user_last_name, song_title)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        session.execute(query3, (user_id, user_first_name, user_last_name, song_title))     

## 4 Selecting the data in the tables to evaluate the complete process

In [10]:
# -------------------------------------------------------------------------------------------------------------- #
# Creating an auxiliar function to transform the query results into a pandas DataFrame
# -------------------------------------------------------------------------------------------------------------- #

def df_from_query(query:str) -> pd.DataFrame:
    """
    Executes a given SELECT query and return the result in a DataFrame
        Args:
            query: Query with a SELECT statement

        Returns:
            pd.DataFrame: A table with the query result
    """
    # sending the query
    rows = session.execute(query)
    
    # getting queried column names
    columns = rows.column_names
    
    # transforming result in a dataframe
    df = pd.DataFrame()
    for row in rows:
        df = df.append(pd.Series(row, columns), ignore_index=True)

    return df


In [11]:
# Query 1: "SELECT artist_name, song_title, song_length FROM sessions WHERE session_id=338 AND item_in_session=4"
query1 = """
    SELECT artist_name
         , song_title
         , song_length
    FROM sessions
    WHERE session_id = 338
          AND item_in_session = 4
"""

df = df_from_query(query1)
df

,artist_name,song_length,song_title
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


In [12]:
# Query 2: "SELECT artist_name, song_title (sorted by item_in_session), user_first_name, user_last_name FROM \
# user_library WHERE user_id=10 AND session_id=182
query2 = """
    SELECT artist_name
         , song_title
         , user_first_name 
         , user_last_name
    FROM user_library
    WHERE user_id = 10
          AND session_id = 182
"""
df = df_from_query(query2)
df

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [13]:
# Query 3: "SELECT user_first_name, user_last_name FROM music_library WHERE song_title='All Hands Against His Own'"
query3 = """
    SELECT user_first_name
         , user_last_name
    FROM music_library
    WHERE song_title = 'All Hands Against His Own'
"""
df = df_from_query(query3)
df

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


## 5 Droping the tables and closing connection

In [14]:
# Dropping all the tables created
queries = ["drop table sessions", "drop table user_library", "drop table music_library"]

for query in queries:
    print(query)
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

drop table sessions
drop table user_library
drop table music_library


### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()